In [ ]:
import pandas as pd
import geopandas as gpd
import unicodedata

df_1 = pd.read_csv('csv/RADIOS SHEET FINAL_data.csv', encoding='utf-8', sep=';')



df_pobreza_radial = df_1[['Código de radio','Departamento','Nivel de incidencia de pobreza crónica', 'Provincia']]

gdf = gpd.read_file('shp_radio_censal/radios2022_v1.0.shp')

df_pobreza_radial = df_pobreza_radial.rename(columns={
    'Código de radio': 'radio_codigo',
    'Departamento': 'departamento_nombre',
    'Nivel de incidencia de pobreza crónica': 'pobreza_cronica_nivel',
    'Provincia': 'provincia_nombre'
})



def normalizar_textos(s):
    if pd.isna(s):
        return ""
    s = s.lower()
    s = "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )
    # Quitar espacios dobles
    s = " ".join(s.split())
    return s

df_pobreza_radial['departamento_nombre'] = df_pobreza_radial['departamento_nombre'].apply(normalizar_textos)
df_pobreza_radial['provincia_nombre'] = df_pobreza_radial['provincia_nombre'].apply(normalizar_textos)

gdf['departament_norm'] = gdf['NOMDEPTO'].apply(normalizar_textos)
gdf['provincia_norm'] = gdf['NOMPROV'].apply(normalizar_textos)



correcciones_dep = {
    "o'higgins": "o higgins",
    "o' higgins": "o higgins",
    'comuna 01': 'comuna 1',
    'comuna 02': 'comuna 2',
    'comuna 03': 'comuna 3',
    'comuna 04': 'comuna 4',
    'comuna 05': 'comuna 5',
    'comuna 06': 'comuna 6',
    'comuna 07': 'comuna 7',
    'comuna 08': 'comuna 8',
    'comuna 09': 'comuna 9',
    'general juan madariaga': 'general madariaga',
    'juan f. ibarra': 'juan felipe ibarra',
    '1� de mayo': '1° de mayo',
    'coronel felipe varela': 'general felipe varela',
    'general ocampo': 'general ortiz de ocampo',
    'general angel v. penaloza': 'angel vicente penaloza',
    "o' higgins": "o'higgins",
    'canuelas': 'caa‘uelas'

}

df_pobreza_radial['departamento_nombre'] = df_pobreza_radial['departamento_nombre'].replace(correcciones_dep)
gdf['departament_norm'] = gdf['departament_norm'].replace(correcciones_dep)

correcciones_bsas = {
    'partidos del agba': 'buenos aires',
    'ciudad de buenos aires': 'caba',
    'buenos aires (sin agba)': 'buenos aires',
    'provincia de buenos aires': 'buenos aires',
    'provincia de buenos aires (sin agba)': 'buenos aires',
    'ciudad autonoma de buenos aires': 'caba'
}
df_pobreza_radial['provincia_nombre'] = df_pobreza_radial['provincia_nombre'].replace(correcciones_bsas)
gdf['provincia_norm'] = gdf['provincia_norm'].replace(correcciones_bsas)


gdf['LINK'] = gdf['LINK'].astype(str)
df_pobreza_radial['radio_codigo'] = df_pobreza_radial['radio_codigo'].astype(str)
gdf['LINK'] = gdf['LINK'].str.lstrip('0')


#Hacemos el merge entre los dos dataframes
gdf = gdf.merge(
    df_pobreza_radial,
    left_on=['LINK', 'provincia_norm'],
    right_on=['radio_codigo', 'provincia_nombre'],
    how='left'
)

#Creamos un map para trabajar mejor con los datos de pobreza
mapeo_pobreza = {
    'Muy bajo (0 - 0,99%)': 0.5,
    'Bajo (1 - 4,99%)': 3.0,
    'Moderado (5 - 9,99%)': 7.5,
    'Alto (10 - 14,99%)': 12.5,
    'Muy alto (15 - 24,99%)': 20.0,
    'Crítico (25 - 100%)': 37.5
}
gdf['pobreza_cronica_nivel'] = gdf['pobreza_cronica_nivel'].map(mapeo_pobreza)
df_pobreza_radial['pobreza_cronica_nivel'] = df_pobreza_radial['pobreza_cronica_nivel'].map(mapeo_pobreza)



     departament_norm provincia_norm      LINK
3976         comuna 4           caba  20281107
Empty DataFrame
Columns: [departamento_nombre, provincia_nombre, radio_codigo]
Index: []


In [ ]:

gdf_caba = (gdf[gdf['provincia_norm'] =='caba'][['provincia_norm', 'departament_norm', 'LINK']])
#print(gdf_caba[gdf_caba['departament_norm']=='comuna 1'])
df_pob_caba = (df_pobreza_radial[df_pobreza_radial['provincia_nombre']=='caba'][['provincia_nombre', 'departamento_nombre', 'radio_codigo']])
#print(df_pob_caba[df_pob_caba['departamento_nombre']=='comuna 1'])
df_pob_caba['radio_codigo'] = df_pob_caba['radio_codigo'].astype(str).str.strip()
gdf_caba['LINK'] = gdf_caba['LINK'].astype(str).str.strip()

print(df_pob_caba['radio_codigo'].str.len().value_counts())
print(gdf_caba['LINK'].str.len().value_counts())

df_pob_caba['radio_codigo'] = df_pob_caba['radio_codigo'].str.zfill(8)
gdf_caba['LINK'] = gdf_caba['LINK'].str.zfill(8)

faltan_en_shp = set(df_pob_caba['radio_codigo']) - set(gdf_caba['LINK'])
faltan_en_df = set(gdf_caba['LINK']) - set(df_pob_caba['radio_codigo'])

print("Códigos en df pero no en shapefile:", list(faltan_en_shp)[:20])
print("Códigos en shapefile pero no en df:", list(faltan_en_df)[:20])

print("No están en shapefile:", len(faltan_en_shp))
print("No están en df:", len(faltan_en_df))


In [3]:
import folium 
from folium.features import GeoJson, GeoJsonTooltip


#Creamos el mapa
map = folium.Map(location=[-38.0, -63.0], zoom_start=5)


#Muy bajo, Bajo, Moderado, Alto, Muy alto, Crítico
colores = ['#1a9850', '#91cf60', '#d9ef8b', '#fee08b', '#fc8d59', '#d73027']


def style_function(pobreza):
    cant_pobreza = pobreza['properties'].get('pobreza_cronica_nivel')
    if cant_pobreza is None:
        return {'fillColor': '#808080', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    try:
        cant_pobreza=float(cant_pobreza)
    except (ValueError, TypeError):
        return {'fillColor': '#808080', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

    if 0 <= cant_pobreza <= 1:
        idx = 0
    elif 1 < cant_pobreza <= 5:
        idx = 1
    elif 5 < cant_pobreza <= 10:
        idx = 2
    elif 10 < cant_pobreza <= 15:
        idx = 3
    elif 15 < cant_pobreza <= 25:
        idx = 4
    elif 25 < cant_pobreza <= 100:
        idx = 5
     
    return {
        'fillColor': colores[idx],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# Añadir GeoJson con tooltip
folium.GeoJson(
    gdf.__geo_interface__,
    style_function=style_function,
    tooltip=GeoJsonTooltip(
        fields=['LINK', 'provincia_norm','departament_norm', 'pobreza_cronica_nivel'],
        aliases=['Codigo censal:', 'Provincia:', 'Departamento:', 'Indice de pobreza:'],
        localize=True
    )
).add_to(map)


#Guardar mapa
map.save('mapas/mapa_pobreza_censal.html')

